In [0]:
%sql
USE CATALOG hive_metastore;
CREATE TABLE managed_default(width INT, length INT, height INT);
INSERT INTO managed_default VALUES(3 INT, 2 INT, 1 INT);


In [0]:
%sql
DESCRIBE EXTENDED managed_default;

In [0]:
%sql
CREATE TABLE external_default(width INT, length INT, height INT)
LOCATION 'dbfs:/mnt/demo/external_default';
INSERT INTO external_default VALUES(3 INT, 2 INT, 1 INT);

In [0]:
%fs
ls 'dbfs:/mnt/demo/external_default'

In [0]:
%sql
DROP TABLE external_default;
DROP TABLE managed_default;

In [0]:
%sql
CREATE SCHEMA new_default;

In [0]:
%sql
DESCRIBE SCHEMA EXTENDED new_default;

In [0]:
%sql
-- CTAS = CREATE TABLE AS SELECT

CREATE TABLE table1 
AS SELECT * FROM table2
/*
CTAS tự động infer schema từ kết quả truy vấn, không hỗ trợ tạo schema thủ công
*/

In [0]:
%sql
-- Filtering and Renaming Columns
/*
Dễ dàng đổi tên cột với CTAS, nếu không đổi tên cột thì cứ SELECT *
*/
CREATE TABLE table_1 
AS SELECT
  col1 as new_col1,
  col2 as new_col2,
  col3
  FROM table_2


In [0]:
%sql
/*
Dễ dàng định nghĩa các comment, partition và location vào trong CTAS
*/
CREATE TABLE table_1
COMMENT "Depzai vcl"
PARTITIONED BY (col1,col2)
LOCATION 'some/way/path'
AS SELECT * FROM table_2

In [0]:
%sql
-- Table Constraint
/*
Delta Lake hỗ trợ ADD CONSTRAINT là NOT NULL và CHECK
Trong trường hợp thiết lập ràng buộc, bắt buộc phải xác định dữ liệu đã có trong bảng không vi phạm ràng buộc nào trước khi xác định ràng buộc
CHECK constraint nó giống như mệnh đề WHERE chuẩn dùng để lọc tập dữ liệu
*/
ALTER TABLE orders ADD CONSTRAINT valid_date CHECK (date > '2021-01-01');

In [0]:
%sql
-- DEEP CLONE
/*
Deep clone - sao chép toàn bộ dữ liệu và metadata từ bảng nguồn sang bảng đích
Nó có thể tự động đồng bộ các thay đổi
Do quá trình ghi dữ liệu từ nguồn lên đích là ghi 1-1 nên sẽ mất khá nhiều thời gian để clone bảng có nhiều dữ liệu
*/
CREATE TABLE clone_tbl DEEP CLONE source_table;

In [0]:
%sql
-- SHALLOW CLONING
/*
Sinh ra để khắc phục sự chậm trễ của deep clone, shallow clone chỉ sao chép Delta transaction logs chứ không sao chép dữ liệu
Là một lựa chọn tốt để thử nghiệm việc áp dụng các thay đổi trên một bảng mà không có nguy cơ sửa đổi bảng
*/
CREATE TABLE shallow_clone_tbl SHALLOW CLONE source_table;

In [0]:
%sql
-- VIEW
/*
DELTA LAKE có đến 3 loại views
- Stored views (view cổ điển)
- Temporary views
- Global Temporary views
*/

-- TEMP VIEW: Là view tạm thời được liên kết với Spark, nó sẽ bị xóa khi SparkSession kết thúc
CREATE TEMP VIEW view_name AS query;
/*
Khi nào thì SparkSession kết thúc
- Tạo một notebook mới
- Hủy kết nối và tái kết nối với một cluster
- Cài đặt thư viện python mới -> python compile cập nhật lại -> SparkSession restart
- Restart cluster
*/

-- Global Temporary view: Khác với temp-view được gắn vào SparkSession thì Global temp view được gắn hẳn vào cluster, miễn là cluster chưa tắt thì vẫn sẽ còn xem được global temp view
CREATE GLOBAL TEMP VIEW view_name AS query;
SELECT global_temp.view_name;



In [0]:
%sql
USE CATALOG hive_metastore;

CREATE TABLE IF NOT EXISTS smartphones (id INT, name STRING, brand STRING, year INT);

INSERT INTO smartphones 
VALUES
  (1, 'iphone 14', 'Apple', 2022),
  (2, 'galaxy s22', 'Samsung', 2022),
  (3, 'pixel 7', 'Google', 2022),
  (4, 'oneplus 10', 'OnePlus', 2022),
  (5, 'xiaomi 12', 'Xiaomi', 2022),
  (6, 'iphone 13', 'Apple', 2021),
  (7, 'galaxy s21', 'Samsung', 2021),
  (8, 'pixel 6', 'Google', 2021),
  (9, 'oneplus 9', 'OnePlus', 2021),
  (10, 'xiaomi 11', 'Xiaomi', 2021);

In [0]:
%sql
-- Stored View
CREATE VIEW view_apple_phones AS SELECT * FROM smartphones WHERE brand = 'Apple';

In [0]:
%sql
SELECT * FROM view_apple_phones;

In [0]:
%sql
UPDATE smartphones SET name='iphone 14 Pro' WHERE id=1;
UPDATE smartphones SET name='iphone 15 Pro' WHERE id=6;

In [0]:
%sql
-- TEMP VIEW
CREATE TEMP VIEW temp_view_phones_brand AS SELECT  DISTINCT brand FROM smartphones;
SELECT * FROM temp_view_phones_brand;

In [0]:
%sql
SHOW TABLES;

In [0]:
%sql
CREATE GLOBAL TEMP VIEW global_temp_view_lastest_phones AS SELECT * FROM smartphones WHERE year = 2021;
SELECT * FROM global_temp.global_temp_view_lastest_phones;

In [0]:
%sql
SHOW TABLES IN global_temp;